In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
import sys
sys.path.append('../')

In [5]:
%load_ext autoreload
%autoreload 2

In [111]:
from Utils_DP.dp_utils import *

In [7]:
from src.pipelines.ingestion import ingest_file

In [8]:
#'../data/incidentes-viales-c5.csv'
data = ingest_file('../data/incidentes-viales-c5.csv')

In [9]:
data.head(3)

,folio,fecha_creacion,hora_creacion,dia_semana,codigo_cierre,fecha_cierre,año_cierre,mes_cierre,hora_cierre,delegacion_inicio,incidente_c4,latitud,longitud,clas_con_f_alarma,tipo_entrada,delegacion_cierre,geopoint,mes
0,GA/160123/05714,23/01/2016,22:35:04,Sábado,(A) La unidad de atención a emergencias fue de...,24/01/2016,2016,Enero,00:21:08,VENUSTIANO CARRANZA,accidente-choque sin lesionados,19.422113,-99.084643,EMERGENCIA,BOTÓN DE AUXILIO,VENUSTIANO CARRANZA,"19.422113,-99.084643",1
1,AO/160123/05826,23/01/2016,22:50:49,Sábado,(A) La unidad de atención a emergencias fue de...,24/01/2016,2016,Enero,04:40:37,CUAJIMALPA,accidente-choque con lesionados,19.358320,-99.297641,URGENCIAS MEDICAS,BOTÓN DE AUXILIO,CUAJIMALPA,"19.35832,-99.297641",1
2,C4/160124/02187,24/01/2016,09:40:11,Domingo,(N) La unidad de atención a emergencias fue de...,24/01/2016,2016,Enero,11:13:03,TLALPAN,accidente-choque sin lesionados,19.217070,-99.219070,EMERGENCIA,LLAMADA DEL 066,TLALPAN,"19.21707,-99.21907008",1


### Generamos análisis primario para observar cómo se comporta la data Raw

In [10]:
count_data(data)

,tipo_dato,cantidad,nulos,unicos,moda,top1,f1,top2,f2,top3,f3
columnas,,,,,,,,,,,
folio,object,1383138,0,1383138,valores unicos,C5/180918/07509,1,AO/160328/00377,1,C4/150329/02722,1
fecha_creacion,object,1383138,0,2637,14/02/2020,14/02/2020,1163,26/10/2018,1028,08/12/2017,1011
hora_creacion,object,1383138,0,105887,20:44:00,20:44:00,80,19:16:00,75,18:38:00,74
dia_semana,object,1383138,0,7,Viernes,Viernes,229681,Sábado,214134,Jueves,198147
codigo_cierre,object,1383138,0,5,(A) La unidad de atención a emergencias fue de...,(A) La unidad de atención a emergencias fue de...,600108,(D) El incidente reportado se registró en dos ...,495631,(N) La unidad de atención a emergencias fue de...,271677
fecha_cierre,object,1383138,0,2621,14/02/2020,14/02/2020,1132,08/12/2017,999,02/12/2017,999
año_cierre,int64,1383138,0,7,2018,2018,246577,2019,229467,2017,226535
mes_cierre,object,1383138,0,12,Octubre,Octubre,133641,Agosto,121123,Septiembre,120581
hora_cierre,object,1383138,0,106827,21:59:00,21:59:00,124,21:52:00,98,21:58:00,96


None

 *********** Informacion global *************


analisis,columnas,datos,total_nulos,%_nulos,num_var,date_var,cat_var,str_var
informacion,18,1383138,1622,0.12%,4,0,0,14


None


#### Haremos algunas trasformaciones a la data antes de hacer Data Profiling como cambiar los tipos de variables a sus correctos.

In [11]:
data_clean = data.copy(deep = True) #Copy deep para que los cambios hechos a data_clean no se vean reflejados en data

Los nombres de las columnas casi están del todo normalizadas para trabajarlas sin espacios y en minúsculas solo la columna **“año_cierre”** por temas de caracteres especiales limpiaremos la **“ñ”** para que quede **“ano_cierre”**

In [12]:
data_clean.columns

Index(['folio', 'fecha_creacion', 'hora_creacion', 'dia_semana',
       'codigo_cierre', 'fecha_cierre', 'año_cierre', 'mes_cierre',
       'hora_cierre', 'delegacion_inicio', 'incidente_c4', 'latitud',
       'longitud', 'clas_con_f_alarma', 'tipo_entrada', 'delegacion_cierre',
       'geopoint', 'mes'],
      dtype='object')

In [13]:
data_clean = clean_column(data_clean)

In [14]:
data_clean.columns

Index(['folio', 'fecha_creacion', 'hora_creacion', 'dia_semana',
       'codigo_cierre', 'fecha_cierre', 'ano_cierre', 'mes_cierre',
       'hora_cierre', 'delegacion_inicio', 'incidente_c4', 'latitud',
       'longitud', 'clas_con_f_alarma', 'tipo_entrada', 'delegacion_cierre',
       'geopoint', 'mes'],
      dtype='object')



En la tabla observamos que las columnas **“ano_cierre”** y  **“mes”** son de tipo numéricas al revisar los datos concluimos que son categorías por lo tanto cambiaremos su tipo a **“String”** para su posterior tratamiento.

In [15]:
data_clean.dtypes

folio                 object
fecha_creacion        object
hora_creacion         object
dia_semana            object
codigo_cierre         object
fecha_cierre          object
ano_cierre             int64
mes_cierre            object
hora_cierre           object
delegacion_inicio     object
incidente_c4          object
latitud              float64
longitud             float64
clas_con_f_alarma     object
tipo_entrada          object
delegacion_cierre     object
geopoint              object
mes                    int64
dtype: object

In [16]:
data_clean = changeType(data_clean,["ano_cierre","mes"],'str')

In [17]:
data_clean.dtypes

folio                 object
fecha_creacion        object
hora_creacion         object
dia_semana            object
codigo_cierre         object
fecha_cierre          object
ano_cierre            object
mes_cierre            object
hora_cierre           object
delegacion_inicio     object
incidente_c4          object
latitud              float64
longitud             float64
clas_con_f_alarma     object
tipo_entrada          object
delegacion_cierre     object
geopoint              object
mes                   object
dtype: object

#### Vemos que existen varias columnas que se relacionan con con las variables fecha y hora de creacion y cierre para facilitar el manejo de estas variables y realizar su Data profiling  las conjuntaremos en una sola columna con el tipo de dato Timestamp.

-

Observamos que las columnas **fecha_creacion** y **fecha_cierre** tienen String en formato **dd/mm/aaaa** y que no tienen valores nulos antes de hacer la conversión a tipo Date hacemos una comprobación de que todos los registros cumplan con este formato.

In [18]:
listFechas = ["fecha_creacion","fecha_cierre"]

In [19]:
checkDateForm(data_clean,listFechas)


Cantidad de valores en fecha_creacion que no cumplen con el formato dd/mm/aaaa : 109047
Distribucion de valores:
 19    186
18      1
Name: 0, dtype: int64

Cantidad de valores en fecha_cierre que no cumplen con el formato dd/mm/aaaa : 109047
Distribucion de valores:
 19    184
Name: 0, dtype: int64


Como podemos observar existen valores que no cumplen con este formato en el año, aplicaremos una transformacion en ambas variables para generalizar este punto.

In [20]:
for i in listFechas:
    data_clean[i] = data_clean[i].replace("/19$", "/2019", regex = True)\
    .replace("/18$", "/2018", regex = True, inplace = False)

Hacemos comprobacion 

In [21]:
checkDateForm(data_clean,listFechas)


Cantidad de valores en fecha_creacion que no cumplen con el formato dd/mm/aaaa : 0
Distribucion de valores:
 Series([], Name: 0, dtype: int64)

Cantidad de valores en fecha_cierre que no cumplen con el formato dd/mm/aaaa : 0
Distribucion de valores:
 Series([], Name: 0, dtype: int64)


-

Observamos que las variables **hora_creacion** y **hora_cierre** tienen strings con el formato hh:mm:ss hagamos una comprobación para saber si todos cumplen con lo mismo.

In [22]:
listHoras = ["hora_creacion","hora_cierre"]

In [23]:
checkTimeHour(data_clean,listHoras)


Cantidad de valores en hora_creacion que no cumplen con el formato hh/mm/ss : 1544
Distribucion de valores:
 0.477615741    2
0.957696759    2
0.787152778    2
0.396122685    2
0.920578704    2
              ..
0.60650463     1
0.636898148    1
0.902361111    1
0.444467593    1
0.402685185    1
Name: hora_creacion, Length: 1528, dtype: int64

Cantidad de valores en hora_cierre que no cumplen con el formato hh/mm/ss : 1542
Distribucion de valores:
 0.767094907    2
0.658657407    2
0.387118056    2
0.854537037    2
0.692893519    2
              ..
0.524108796    1
0.142083333    1
0.593402778    1
0.925335648    1
0.419780093    1
Name: hora_cierre, Length: 1519, dtype: int64


Como podemos observar existen valores que no cumplen con este formato, aplicaremos una transformacion en ambas variables para generalizar este punto haremos una transformacion con pd_deltatime.

In [24]:
for i in listHoras:
    z = data_clean[data_clean[i].str.match('^([01]?[0-9]|2[0-3]):[0-5][0-9]:[0-5][0-9]$')==False]
    data_clean.loc[z.index,i] = pd.to_timedelta(z[i].astype(float),'days')

Hacemos comprobación 

In [25]:
checkTimeHour(data_clean,listHoras)


Cantidad de valores en hora_creacion que no cumplen con el formato hh/mm/ss : 0
Distribucion de valores:
 Series([], Name: hora_creacion, dtype: int64)

Cantidad de valores en hora_cierre que no cumplen con el formato hh/mm/ss : 0
Distribucion de valores:
 Series([], Name: hora_cierre, dtype: int64)


Ahora uniremos estas columnas de fecha y hora respectivamente para creacion y otra para cierre con el tipo datetime.

In [31]:
data_clean['dttm_creacion'] = pd.to_datetime(data_clean["fecha_creacion"],format ='%d/%m/%Y')+pd.to_timedelta(data_clean["hora_creacion"],'ns')
data_clean['dttm_cierre'] = pd.to_datetime(data_clean["fecha_cierre"],format ='%d/%m/%Y')+pd.to_timedelta(data_clean["hora_cierre"],'ns')

Hacemos comprobación de tipos de datos y si existen nulos en esta transformación.

In [33]:
data_clean.dtypes

folio                        object
fecha_creacion               object
hora_creacion                object
dia_semana                   object
codigo_cierre                object
fecha_cierre                 object
ano_cierre                   object
mes_cierre                   object
hora_cierre                  object
delegacion_inicio            object
incidente_c4                 object
latitud                     float64
longitud                    float64
clas_con_f_alarma            object
tipo_entrada                 object
delegacion_cierre            object
geopoint                     object
mes                          object
dttm_creacion        datetime64[ns]
dttm_cierre          datetime64[ns]
dtype: object

In [34]:
data_clean.isnull().sum()

folio                  0
fecha_creacion         0
hora_creacion          0
dia_semana             0
codigo_cierre          0
fecha_cierre           0
ano_cierre             0
mes_cierre             0
hora_cierre            0
delegacion_inicio    158
incidente_c4           0
latitud              443
longitud             435
clas_con_f_alarma      0
tipo_entrada           0
delegacion_cierre    140
geopoint             446
mes                    0
dttm_creacion          0
dttm_cierre            0
dtype: int64

Observamos que no hubo perdida de datos con estas transformaciones ahora procederemos a borrar las columnas de creacion y cierre tanto de fecha como hora para trabajar con las creadas en tipo datetime.

In [35]:
data_clean.drop(columns = ['fecha_creacion', 'hora_creacion','fecha_cierre','hora_cierre'], inplace = True)

In [36]:
data_clean.isnull().sum()

folio                  0
dia_semana             0
codigo_cierre          0
ano_cierre             0
mes_cierre             0
delegacion_inicio    158
incidente_c4           0
latitud              443
longitud             435
clas_con_f_alarma      0
tipo_entrada           0
delegacion_cierre    140
geopoint             446
mes                    0
dttm_creacion          0
dttm_cierre            0
dtype: int64

Con esto concluimos la asignacion de tipos corrector y creacion de variables con fecha y hora procedemos a realizar el Data Profiling. 

# Data Profiling 

In [37]:
data_clean.dtypes

folio                        object
dia_semana                   object
codigo_cierre                object
ano_cierre                   object
mes_cierre                   object
delegacion_inicio            object
incidente_c4                 object
latitud                     float64
longitud                    float64
clas_con_f_alarma            object
tipo_entrada                 object
delegacion_cierre            object
geopoint                     object
mes                          object
dttm_creacion        datetime64[ns]
dttm_cierre          datetime64[ns]
dtype: object

Obtenemos las variables con las que trabajaremos en el data profiling.

In [38]:
dicVar = getVariables(data_clean)

In [39]:
dicVar

{'num_var': (2, array(['latitud', 'longitud'], dtype=object)),
 'date_var': (2, array(['dttm_creacion', 'dttm_cierre'], dtype=object)),
 'cat_var': (0, array([], dtype=object)),
 'str_var': (12,
  array(['folio', 'dia_semana', 'codigo_cierre', 'ano_cierre', 'mes_cierre',
         'delegacion_inicio', 'incidente_c4', 'clas_con_f_alarma',
         'tipo_entrada', 'delegacion_cierre', 'geopoint', 'mes'],
        dtype=object))}

### Variables Categoricas

In [115]:
data_profiling_categ(data_clean,dicVar['str_var'][1])

metrica,folio,dia_semana,codigo_cierre,ano_cierre,mes_cierre,delegacion_inicio,incidente_c4,clas_con_f_alarma,tipo_entrada,delegacion_cierre,geopoint,mes
registros,1383138,1383138,1383138,1383138,1383138,1383138,1383138,1383138,1383138,1383138,1383138,1383138
nulos,0,0,0,0,0,158,0,0,0,140,446,0
Num_categorias,1383138,7,5,7,12,16,26,4,9,16,134288,12
categorias,"[GA/160123/05714, AO/160123/05826, C4/160124/0...","[Sábado, Domingo, Lunes, Martes, Miércoles, Ju...",[(A) La unidad de atención a emergencias fue d...,"[2016, 2017, 2018, 2019, 2015, 2014, 2020]","[Enero, Febrero, Marzo, Septiembre, Octubre, M...","[VENUSTIANO CARRANZA, CUAJIMALPA, TLALPAN, MAG...","[accidente-choque sin lesionados, accidente-ch...","[EMERGENCIA, URGENCIAS MEDICAS, FALSA ALARMA, ...","[BOTÓN DE AUXILIO, LLAMADA DEL 066, RADIO, CÁM...","[VENUSTIANO CARRANZA, CUAJIMALPA, TLALPAN, MAG...","[19.422113,-99.084643, 19.35832,-99.297641, 19...","[1, 2, 3, 9, 10, 5, 6, 4, 8, 12, 7, 11]"
moda,AO/140101/01144,Viernes,(A) La unidad de atención a emergencias fue de...,2018,Octubre,IZTAPALAPA,accidente-choque sin lesionados,EMERGENCIA,LLAMADA DEL 911,IZTAPALAPA,"19.37168001,-99.08024004",10
top1,C5/180918/07509,Viernes,(A) La unidad de atención a emergencias fue de...,2018,Octubre,IZTAPALAPA,accidente-choque sin lesionados,EMERGENCIA,LLAMADA DEL 911,IZTAPALAPA,"19.37168001,-99.08024004",10
f1,1,229681,600108,246577,133641,223687,760506,799037,739339,225202,2162,133641
top2,AO/160328/00377,Sábado,(D) El incidente reportado se registró en dos ...,2019,Agosto,GUSTAVO A. MADERO,accidente-choque con lesionados,URGENCIAS MEDICAS,LLAMADA DEL 066,GUSTAVO A. MADERO,"19.30431996,-99.08714016",8
f2,1,214134,495631,229467,121123,144620,315031,572619,463432,147582,2094,121123
top3,C4/150329/02722,Jueves,(N) La unidad de atención a emergencias fue de...,2017,Septiembre,CUAUHTEMOC,lesionado-atropellado,FALSA ALARMA,BOTÓN DE AUXILIO,CUAUHTEMOC,"19.347021,-99.180646",9


### Variables Numericas

In [62]:
data_profiling_num(data_clean,dicVar['num_var'][1])

metricas,latitud,longitud
cantidad,1.382695e+06,1.382703e+06
nulos,4.430000e+02,4.350000e+02
unicos,8.250100e+04,7.898400e+04
moda,1.930432e+01,-9.908024e+01
min,1.909402e+01,-9.917640e+02
max,1.953030e+02,-9.894537e+01
mean,1.938391e+01,-9.914359e+01
stdv,2.666382e-01,2.399677e+00
25%,1.933685e+01,-9.917930e+01
median,1.938408e+01,-9.914023e+01


### Variables de Fechas

In [114]:
data_profiling_date(data_clean,dicVar['date_var'][1])

metricas,dttm_creacion,dttm_cierre
tipo dato,<class 'numpy.datetime64'>,<class 'numpy.datetime64'>
registros,1383138,1383138
nulos,0,0
unicos,1367004,1369644
moda,0 2018-08-22 08:53:00 dtype: datetime64[ns],0 2019-11-06 18:39:00 dtype: datetime64[ns]
fecha inicio,2013-12-31 13:01:33,2014-01-01 00:00:03
fecha fin,2020-10-31 23:57:20,2020-10-31 23:59:04
top1,2018-08-22 08:53:00,2019-11-06 18:39:00
f1,10,8
top2,2018-01-04 20:41:00,2019-03-23 06:07:00


In [ ]:
delegacion inicio 
incidente c4
latitud y longitud 
tipo de entreda y ddtime de craeacion 

In [ ]:
folio 
delegacion cierre no la tendriamos al momento d ela llamda 
clasificacion con falsa alarma no la tendrimos al momento de la llamada 
delegacion cierre no la tendriamos al moento de la llamada 
fecha de cierre tampoco la tenmdriamos